# Introduction

## Objectif du Notebook
L'objectif de ce notebook est de développer et d'évaluer des modèles avancés d'analyse de sentiments en utilisant des architectures de réseaux de neurones convolutionnels (CNN) et de réseaux de neurones récurrents (LSTM). Ces modèles seront entraînés sur un jeu de données de tweets et utiliseront des embeddings textuels pré-entraînés tels que Word2Vec et FastText pour capturer les relations sémantiques entre les mots.

## Présentation des Méthodes CNN et LSTM
Le Convolutional Neural Network (CNN) et le Long Short-Term Memory (LSTM) sont deux types d'architectures de réseaux de neurones particulièrement adaptées au traitement des données séquentielles telles que le texte.

### CNN (Convolutional Neural Network)
Les CNN sont bien connus pour leur capacité à détecter des motifs locaux dans les données séquentielles, ce qui les rend efficaces pour capturer des n-grams de mots dans les textes. Dans ce notebook, nous utiliserons un CNN pour analyser les séquences de mots des tweets et en extraire des caractéristiques pertinentes pour la classification de sentiments.

### LSTM (Long Short-Term Memory)
Les LSTM sont un type de réseau de neurones récurrents (RNN) capable de capturer des dépendances à long terme dans des séquences de données. Cela les rend particulièrement efficaces pour comprendre le contexte complet d'une phrase ou d'un tweet, en tenant compte de l'ordre des mots et des relations temporelles.

## Plan du Notebook
Ce notebook est structuré de manière à suivre un pipeline complet de traitement des données, d'entraînement des modèles et d'évaluation des performances :
1. **Chargement et Préparation des Données** : Nous allons commencer par charger les données, les nettoyer, les tokeniser et les préparer pour les modèles CNN et LSTM.
2. **Préparation des Embeddings Textuels** : Nous utiliserons Word2Vec et FastText pour convertir les séquences de mots en vecteurs denses.
3. **Construction du Modèle CNN** : Nous définirons et entraînerons un modèle CNN pour la tâche de classification de sentiments.
4. **Construction du Modèle LSTM** : Nous définirons et entraînerons un modèle LSTM pour la même tâche.
5. **Évaluation des Modèles** : Nous comparerons les performances des deux modèles pour déterminer lequel est le plus performant.
6. **Exportation et Sauvegarde des Modèles** : Nous sauvegarderons le meilleur modèle pour une utilisation future.
7. **Conclusion** : Nous résumerons les résultats et discuterons des perspectives d'amélioration.


# 1. Chargement et Préparation des Données

## 1.1 Importation des bibliothèques nécessaires


In [4]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


## 1.2 Chargement des données et transformation de la cible target


In [6]:
# Charger les données
data = pd.read_csv('../data/database_p7_rework.csv')

# Transformation des labels : 0 reste 0 et 4 devient 1
data['target_binary'] = data['target'].apply(lambda x: 0 if x == 0 else 1)

# Vérification des transformations
print("Unique values in target_binary:", data['target_binary'].unique())



Unique values in target_binary: [0 1]


## 1.3 Tokenisation

In [8]:
# Fonction pour préparer les séquences
def prepare_sequences(texts, tokenizer, maxlen=25):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=maxlen)

# Tokenization
tokenizer_lemma = Tokenizer(num_words=5000)
tokenizer_lemma.fit_on_texts(data['text_lemmatized'])

tokenizer_stem = Tokenizer(num_words=5000)
tokenizer_stem.fit_on_texts(data['text_stemmed'])

# Préparation des séquences
X_lemma = prepare_sequences(data['text_lemmatized'], tokenizer_lemma)
X_stem = prepare_sequences(data['text_stemmed'], tokenizer_stem)

# Séparation des données
y = data['target_binary']

X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(X_lemma, y, test_size=0.2, random_state=42)
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(X_stem, y, test_size=0.2, random_state=42)

# 2 Préparation des Embeddings Textuels

In [10]:
# Charger les modèles Word2Vec et FastText pré-entraînés (300 dimensions)
w2v_model = api.load('word2vec-google-news-300')
ft_model = api.load('fasttext-wiki-news-subwords-300')

# Fonction pour créer la matrice d'embeddings
def create_embedding_matrix(tokenizer, embedding_model, embedding_dim=300):
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
    return embedding_matrix

# Créer les matrices d'embeddings
embedding_matrix_lemma_w2v = create_embedding_matrix(tokenizer_lemma, w2v_model)
embedding_matrix_stem_w2v = create_embedding_matrix(tokenizer_stem, w2v_model)

embedding_matrix_lemma_ft = create_embedding_matrix(tokenizer_lemma, ft_model)
embedding_matrix_stem_ft = create_embedding_matrix(tokenizer_stem, ft_model)


# 3. Définition de l'architecture du modèle CNN

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Fonction pour créer un modèle CNN
def create_cnn_model(input_dim, embedding_matrix, max_length=100):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Conv1D(128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model




# 4. Définition de l'architecture du modèle LSTM

In [14]:
from keras.layers import LSTM, Bidirectional

# Fonction pour créer un modèle LSTM
def create_lstm_model(input_dim, embedding_matrix, max_length=100):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



# 5. Entraînement et Évaluation des Modèles

In [42]:
import mlflow
import os

# Définir le dossier racine où vous voulez enregistrer les artefacts MLflow
base_dir = os.path.join("..", "mlruns")

# Liste des modèles que vous allez entraîner
model_list = [
    "cnn_model_lemma_w2v",
    "cnn_model_lemma_ft",
    "cnn_model_stem_w2v",
    "cnn_model_stem_ft",
    "lstm_model_lemma_w2v",
    "lstm_model_lemma_ft",
    "lstm_model_stem_w2v",
    "lstm_model_stem_ft",
]

for model_name in model_list:
    # Créer le chemin complet pour ce modèle spécifique
    mlflow_base_dir = os.path.join(base_dir, model_name)

    # Créer le dossier si nécessaire
    if not os.path.exists(mlflow_base_dir):
        os.makedirs(mlflow_base_dir)

    # Configurer MLflow pour utiliser ce dossier comme URI de suivi
    mlflow.set_tracking_uri(f"file:///{mlflow_base_dir}")



## 5.1 Entraînement des Modèles CNN et LSTM avec les Embeddings Word2Vec + lemmatisation

In [ ]:
# Entraînement et évaluation pour chaque combinaison (lemmatisation/stemming et Word2Vec/FastText)
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
# CNN avec lemmatisation + Word2Vec
with mlflow.start_run():
    cnn_model_lemma_w2v = create_cnn_model(len(tokenizer_lemma.word_index) + 1, embedding_matrix_lemma_w2v)
    cnn_model_lemma_w2v.fit(X_train_lemma, y_train_lemma, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(cnn_model_lemma_w2v, "cnn_model_lemma_w2v")
    y_pred_lemma_w2v = (cnn_model_lemma_w2v.predict(X_test_lemma) > 0.5).astype("int32")
    accuracy_lemma_w2v = accuracy_score(y_test_lemma, y_pred_lemma_w2v)
    mlflow.log_metric("accuracy_lemma_w2v", accuracy_lemma_w2v)
    cm_lemma_w2v = confusion_matrix(y_test_lemma, y_pred_lemma_w2v)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lemma_w2v, annot=True, fmt='d')
    plt.title("Confusion Matrix: CNN + Lemmatization + Word2Vec")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# LSTM avec lemmatisation + Word2Vec
with mlflow.start_run():
    lstm_model_lemma_w2v = create_lstm_model(len(tokenizer_lemma.word_index) + 1, embedding_matrix_lemma_w2v)
    lstm_model_lemma_w2v.fit(X_train_lemma, y_train_lemma, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(lstm_model_lemma_w2v, "lstm_model_lemma_w2v")
    y_pred_lstm_lemma_w2v = (lstm_model_lemma_w2v.predict(X_test_lemma) > 0.5).astype("int32")
    accuracy_lstm_lemma_w2v = accuracy_score(y_test_lemma, y_pred_lstm_lemma_w2v)
    mlflow.log_metric("accuracy_lstm_lemma_w2v", accuracy_lstm_lemma_w2v)
    cm_lstm_lemma_w2v = confusion_matrix(y_test_lemma, y_pred_lstm_lemma_w2v)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lstm_lemma_w2v, annot=True, fmt='d')
    plt.title("Confusion Matrix: LSTM + Lemmatization + Word2Vec")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

D:\Master_Openclassroom\python\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 50s 3ms/step - accuracy: 0.7744 - loss: 0.4704 - val_accuracy: 0.8025 - val_loss: 0.4255
Epoch 2/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.8089 - loss: 0.4148 - val_accuracy: 0.8067 - val_loss: 0.4189
Epoch 3/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.8193 - loss: 0.3959 - val_accuracy: 0.8078 - val_loss: 0.4166
Epoch 4/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 50s 3ms/step - accuracy: 0.8272 - loss: 0.3814 - val_accuracy: 0.8091 - val_loss: 0.4158
Epoch 5/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.8322 - loss: 0.3702 - val_accuracy: 0.8085 - val_loss: 0.4202
Epoch 6/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 54s 3ms/step - accuracy: 0.8386 - loss: 0.3591 - val_accuracy: 0.8045 - val_loss: 0.4254
Epoch 7/10
15984/15984 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - accuracy: 0.8427 - loss: 0.3507 - val_accuracy: 0.8062 - val_loss: 0.4310
Epoch 8/10
 7131/15984 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 

## 5.2 Entraînement des Modèles CNN et LSTM avec les Embeddings FastText + lemmatisation

In [ ]:
# CNN avec lemmatisation + FastText
with mlflow.start_run():
    cnn_model_lemma_ft = create_cnn_model(len(tokenizer_lemma.word_index) + 1, embedding_matrix_lemma_ft)
    cnn_model_lemma_ft.fit(X_train_lemma, y_train_lemma, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(cnn_model_lemma_ft, "cnn_model_lemma_ft")
    y_pred_lemma_ft = (cnn_model_lemma_ft.predict(X_test_lemma) > 0.5).astype("int32")
    accuracy_lemma_ft = accuracy_score(y_test_lemma, y_pred_lemma_ft)
    mlflow.log_metric("accuracy_lemma_ft", accuracy_lemma_ft)
    cm_lemma_ft = confusion_matrix(y_test_lemma, y_pred_lemma_ft)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lemma_ft, annot=True, fmt='d')
    plt.title("Confusion Matrix: CNN + Lemmatization + FastText")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# LSTM avec lemmatisation + FastText
with mlflow.start_run():
    lstm_model_lemma_ft = create_lstm_model(len(tokenizer_lemma.word_index) + 1, embedding_matrix_lemma_ft)
    lstm_model_lemma_ft.fit(X_train_lemma, y_train_lemma, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(lstm_model_lemma_ft, "lstm_model_lemma_ft")
    y_pred_lstm_lemma_ft = (lstm_model_lemma_ft.predict(X_test_lemma) > 0.5).astype("int32")
    accuracy_lstm_lemma_ft = accuracy_score(y_test_lemma, y_pred_lstm_lemma_ft)
    mlflow.log_metric("accuracy_lstm_lemma_ft", accuracy_lstm_lemma_ft)
    cm_lstm_lemma_ft = confusion_matrix(y_test_lemma, y_pred_lstm_lemma_ft)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lstm_lemma_ft, annot=True, fmt='d')
    plt.title("Confusion Matrix: LSTM + Lemmatization + FastText")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()


## 5.3 Entraînement des Modèles CNN et LSTM avec les Embeddings Word2Vec + stemming

In [ ]:
# CNN avec stemming + Word2Vec
with mlflow.start_run():
    cnn_model_stem_w2v = create_cnn_model(len(tokenizer_stem.word_index) + 1, embedding_matrix_stem_w2v)
    cnn_model_stem_w2v.fit(X_train_stem, y_train_stem, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(cnn_model_stem_w2v, "cnn_model_stem_w2v")
    y_pred_stem_w2v = (cnn_model_stem_w2v.predict(X_test_stem) > 0.5).astype("int32")
    accuracy_stem_w2v = accuracy_score(y_test_stem, y_pred_stem_w2v)
    mlflow.log_metric("accuracy_stem_w2v", accuracy_stem_w2v)
    cm_stem_w2v = confusion_matrix(y_test_stem, y_pred_stem_w2v)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_stem_w2v, annot=True, fmt='d')
    plt.title("Confusion Matrix: CNN + Stemming + Word2Vec")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# LSTM avec stemming + Word2Vec
with mlflow.start_run():
    lstm_model_stem_w2v = create_lstm_model(len(tokenizer_stem.word_index) + 1, embedding_matrix_stem_w2v)
    lstm_model_stem_w2v.fit(X_train_stem, y_train_stem, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(lstm_model_stem_w2v, "lstm_model_stem_w2v")
    y_pred_lstm_stem_w2v = (lstm_model_stem_w2v.predict(X_test_stem) > 0.5).astype("int32")
    accuracy_lstm_stem_w2v = accuracy_score(y_test_stem, y_pred_lstm_stem_w2v)
    mlflow.log_metric("accuracy_lstm_stem_w2v", accuracy_lstm_stem_w2v)
    cm_lstm_stem_w2v = confusion_matrix(y_test_stem, y_pred_lstm_stem_w2v)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lstm_stem_w2v, annot=True, fmt='d')
    plt.title("Confusion Matrix: LSTM + Stemming + Word2Vec")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

## 5.4 Entraînement des Modèles CNN et LSTM avec les Embeddings FastText + stemming

In [ ]:
# CNN avec stemming + FastText
with mlflow.start_run():
    cnn_model_stem_ft = create_cnn_model(len(tokenizer_stem.word_index) + 1, embedding_matrix_stem_ft)
    cnn_model_stem_ft.fit(X_train_stem, y_train_stem, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(cnn_model_stem_ft, "cnn_model_stem_ft")
    y_pred_stem_ft = (cnn_model_stem_ft.predict(X_test_stem) > 0.5).astype("int32")
    accuracy_stem_ft = accuracy_score(y_test_stem, y_pred_stem_ft)
    mlflow.log_metric("accuracy_stem_ft", accuracy_stem_ft)
    cm_stem_ft = confusion_matrix(y_test_stem, y_pred_stem_ft)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_stem_ft, annot=True, fmt='d')
    plt.title("Confusion Matrix: CNN + Stemming + FastText")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# LSTM avec stemming + FastText
with mlflow.start_run():
    lstm_model_stem_ft = create_lstm_model(len(tokenizer_stem.word_index) + 1, embedding_matrix_stem_ft)
    lstm_model_stem_ft.fit(X_train_stem, y_train_stem, batch_size=64, epochs=10, validation_split=0.2, verbose=1)
    mlflow.keras.log_model(lstm_model_stem_ft, "lstm_model_stem_ft")
    y_pred_lstm_stem_ft = (lstm_model_stem_ft.predict(X_test_stem) > 0.5).astype("int32")
    accuracy_lstm_stem_ft = accuracy_score(y_test_stem, y_pred_lstm_stem_ft)
    mlflow.log_metric("accuracy_lstm_stem_ft", accuracy_lstm_stem_ft)
    cm_lstm_stem_ft = confusion_matrix(y_test_stem, y_pred_lstm_stem_ft)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm_lstm_stem_ft, annot=True, fmt='d')
    plt.title("Confusion Matrix: LSTM + Stemming + FastText")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# 6. Comparaison des modèles

In [ ]:
import pandas as pd

# Initialiser une liste pour stocker les résultats
results = []

# Fonction pour ajouter les résultats d'un modèle à la liste
def log_results(model_name, accuracy, precision, recall, f1):
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# Log des résultats pour les modèles avancés (Word2Vec + FastText)

# CNN + Lemmatization + Word2Vec
log_results(
    'CNN + Lemmatization + Word2Vec',
    accuracy_cnn_lemma_w2v,
    precision_cnn_lemma_w2v,
    recall_cnn_lemma_w2v,
    f1_cnn_lemma_w2v
)

# LSTM + Lemmatization + Word2Vec
log_results(
    'LSTM + Lemmatization + Word2Vec',
    accuracy_lstm_lemma_w2v,
    precision_lstm_lemma_w2v,
    recall_lstm_lemma_w2v,
    f1_lstm_lemma_w2v
)

# CNN + Stemming + Word2Vec
log_results(
    'CNN + Stemming + Word2Vec',
    accuracy_cnn_stem_w2v,
    precision_cnn_stem_w2v,
    recall_cnn_stem_w2v,
    f1_cnn_stem_w2v
)

# LSTM + Stemming + Word2Vec
log_results(
    'LSTM + Stemming + Word2Vec',
    accuracy_lstm_stem_w2v,
    precision_lstm_stem_w2v,
    recall_lstm_stem_w2v,
    f1_lstm_stem_w2v
)

# CNN + Lemmatization + FastText
log_results(
    'CNN + Lemmatization + FastText',
    accuracy_cnn_lemma_ft,
    precision_cnn_lemma_ft,
    recall_cnn_lemma_ft,
    f1_cnn_lemma_ft
)

# LSTM + Lemmatization + FastText
log_results(
    'LSTM + Lemmatization + FastText',
    accuracy_lstm_lemma_ft,
    precision_lstm_lemma_ft,
    recall_lstm_lemma_ft,
    f1_lstm_lemma_ft
)

# CNN + Stemming + FastText
log_results(
    'CNN + Stemming + FastText',
    accuracy_cnn_stem_ft,
    precision_cnn_stem_ft,
    recall_cnn_stem_ft,
    f1_cnn_stem_ft
)

# LSTM + Stemming + FastText
log_results(
    'LSTM + Stemming + FastText',
    accuracy_lstm_stem_ft,
    precision_lstm_stem_ft,
    recall_lstm_stem_ft,
    f1_lstm_stem_ft
)

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Afficher le tableau récapitulatif
print(results_df)

